In [1]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import numpy as np
from stravaTokenManager import refreshToken
import time


In [3]:
load_dotenv()

users = ["", "F1", "F2", "F3"]

for user in users:
    refreshToken(
        client_id=os.getenv(f"STRAVA_{user}_CLIENT_ID"),
        client_secret=os.getenv(f"STRAVA_{user}_CLIENT_SECRET"),
        refresh_token=os.getenv(f"STRAVA_{user}_REFRESH_TOKEN"),
        user_prefix=f"STRAVA_{user}"
    )


[STRAVA_] Tokens Refreshed Successfully.
[STRAVA_] Tokens Successfully Updated In .env File.
[STRAVA_F1] Tokens Refreshed Successfully.
[STRAVA_F1] Tokens Successfully Updated In .env File.
[STRAVA_F2] Tokens Refreshed Successfully.
[STRAVA_F2] Tokens Successfully Updated In .env File.
[STRAVA_F3] Tokens Refreshed Successfully.
[STRAVA_F3] Tokens Successfully Updated In .env File.


In [5]:
load_dotenv(override=True)
accessTokenMyself = os.getenv("STRAVA__ACCESS_TOKEN")
accessTokenF1 = os.getenv("STRAVA_F1_ACCESS_TOKEN")
accessTokenF2 = os.getenv("STRAVA_F2_ACCESS_TOKEN")
accessTokenF3 = os.getenv("STRAVA_F3_ACCESS_TOKEN")

In [41]:
def get_strava_data(access_token, delay=1):
    all_activities = []
    page = 1
    while True:
        response = requests.get(
            "https://www.strava.com/api/v3/athlete/activities",
            headers={"Authorization": f"Bearer {access_token}"},
            params={"per_page": 200, "page": page}
        )

        if response.status_code == 429:
            print("Rate limit hit. Waiting 15 minutes...")
            time.sleep(15 * 60)
            continue

        data = response.json()

        if not data or "message" in data:
            break

        all_activities.extend(data)
        print(f"Page {page} fetched. {len(data)} activities")
        
        page += 1
        time.sleep(delay)

    print(f"Collection Complete.Total: {page} pages and {len(all_activities)} activities.")

    return all_activities


In [9]:
my_runs = get_strava_data(accessTokenMyself)

In [43]:
f1_runs = get_strava_data(accessTokenF1)

Page 1 fetched. 75 activities
Collection Complete.Total: 2 pages and 75 activities.


In [45]:
f2_runs = get_strava_data(accessTokenF2)

Page 1 fetched. 200 activities
Page 2 fetched. 200 activities
Page 3 fetched. 20 activities
Collection Complete.Total: 4 pages and 420 activities.


In [35]:
f3_runs = get_strava_data(accessTokenF3)

Page 1 fetched. 49 activities
Collection Complete. Total: 2 pages and 0 activities.


In [21]:
cols = ['id',
        'start_date_local',
        'distance',
        'moving_time',
        'total_elevation_gain',
        'type',
        'average_speed',
        'max_speed']

In [47]:
my_runs = pd.DataFrame(my_runs)[cols]

In [49]:
f1_runs = pd.DataFrame(f1_runs)[cols]

In [51]:
f2_runs = pd.DataFrame(f2_runs)[cols]

In [53]:
f3_runs = pd.DataFrame(f3_runs)[cols]

In [55]:
def filter2025(df, date_col = 'start_date_local', year_of_interest = 2025):
    newdf = df.copy()
    newdf[date_col] = pd.to_datetime(newdf[date_col])
    newdf = newdf[newdf[date_col].dt.year == year_of_interest]
    return newdf

In [57]:
my_runs = filter2025(my_runs)

In [59]:
my_runs['friendNum'] = np.zeros(my_runs.shape[0], dtype = int)

In [61]:
my_runs

,id,start_date_local,distance,moving_time,total_elevation_gain,type,average_speed,max_speed,friendNum
0,15193983776,2025-07-21 19:23:22+00:00,8103.1,3258,65.6,Run,2.487,3.880,0
1,15186974746,2025-07-20 20:11:27+00:00,8059.0,2608,41.6,Run,3.090,6.140,0
2,15150887333,2025-07-17 19:57:52+00:00,6489.2,1981,37.0,Run,3.276,6.311,0
3,15128512859,2025-07-15 17:56:13+00:00,6156.5,2260,34.6,Run,2.724,4.460,0
4,15122419532,2025-07-14 22:17:49+00:00,1434.1,1018,6.4,Walk,1.409,2.293,0
...,...,...,...,...,...,...,...,...,...
94,13258123522,2025-01-03 12:21:04+00:00,1268.1,834,13.2,Walk,1.520,2.493,0
95,13251708766,2025-01-02 17:54:59+00:00,2745.1,1674,46.8,Walk,1.640,4.680,0
96,13251500543,2025-01-02 16:47:51+00:00,8056.3,3116,45.4,Run,2.585,5.220,0
97,13240756945,2025-01-01 10:23:14+00:00,2651.8,1026,31.6,Run,2.585,4.140,0


In [63]:
f1_runs = filter2025(f1_runs)

In [65]:
f1_runs['friendNum'] = np.ones(f1_runs.shape[0], dtype = int)

In [67]:
f1_runs

,id,start_date_local,distance,moving_time,total_elevation_gain,type,average_speed,max_speed,friendNum
0,14896650906,2025-06-23 19:38:02+00:00,5565.9,3083,15.8,Walk,1.805,5.72,1
1,14780241420,2025-06-12 11:51:55+00:00,4138.1,2969,11.3,Walk,1.394,3.70,1
2,14779728419,2025-06-12 11:43:53+00:00,1340.8,452,4.5,Run,2.966,4.23,1
3,14678651086,2025-06-02 18:11:33+00:00,5577.9,3239,15.7,Walk,1.722,4.28,1
4,14637244057,2025-05-29 18:34:27+00:00,5533.7,3448,15.9,Walk,1.605,6.98,1
5,14510465933,2025-05-17 06:35:56+00:00,18124.2,6817,233.9,Run,2.659,4.68,1
6,14508436611,2025-05-17 06:14:17+00:00,1093.2,665,0.0,Walk,1.644,4.04,1
7,14398166848,2025-05-06 11:12:10+00:00,9555.0,3440,20.1,Run,2.778,4.44,1
8,13791902511,2025-03-04 16:16:50+00:00,5577.8,1850,17.0,Run,3.015,4.68,1
9,13720480779,2025-02-24 23:38:14+00:00,3453.6,2104,7.4,Walk,1.641,5.60,1


In [69]:
f2_runs = filter2025(f2_runs)

In [71]:
f2_runs['friendNum'] = np.ones(f2_runs.shape[0], dtype = int) * 2

In [73]:
f2_runs

,id,start_date_local,distance,moving_time,total_elevation_gain,type,average_speed,max_speed,friendNum
0,15190395019,2025-07-21 12:09:39+00:00,2074.2,1840,34.4,Walk,1.127,2.240,2
1,15160713668,2025-07-18 18:55:10+00:00,2015.9,1613,13.3,Walk,1.250,3.813,2
2,15160585207,2025-07-18 18:42:12+00:00,2427.5,734,9.2,Run,3.307,5.293,2
3,15150587489,2025-07-17 18:07:10+00:00,10145.0,4000,87.3,Run,2.536,5.460,2
4,15139866091,2025-07-16 05:24:40+00:00,802.3,750,9.3,Walk,1.070,1.900,2
...,...,...,...,...,...,...,...,...,...
169,13256955840,2025-01-03 09:24:37+00:00,3686.0,1453,18.8,Run,2.537,3.960,2
170,13252424135,2025-01-02 21:01:28+00:00,2418.3,1823,0.0,Walk,1.327,2.400,2
171,13243229253,2025-01-01 19:34:52+00:00,3228.0,2503,0.0,Walk,1.290,2.412,2
172,13240789689,2025-01-01 10:48:04+00:00,808.4,778,4.4,Walk,1.039,2.213,2


In [75]:
f3_runs = filter2025(f3_runs)

In [77]:
f3_runs['friendNum'] = np.ones(f3_runs.shape[0], dtype = int) * 3

In [79]:
f3_runs

,id,start_date_local,distance,moving_time,total_elevation_gain,type,average_speed,max_speed,friendNum
0,15193449543,2025-07-21 19:23:28+00:00,8116.8,3276,61.5,Run,2.478,4.700,3
1,15156872045,2025-07-18 09:39:24+00:00,4286.5,1193,24.8,Run,3.593,5.240,3
2,14700898722,2025-06-04 19:59:25+00:00,15065.5,4740,76.1,Run,3.178,6.500,3
3,14653194561,2025-05-31 07:41:59+00:00,5339.8,2898,25.1,Run,1.843,4.020,3
4,14653194530,2025-05-29 17:10:47+00:00,3165.1,1580,12.2,Run,2.003,5.020,3
5,14626922762,2025-05-28 18:57:47+00:00,10472.1,4234,59.3,Run,2.473,4.680,3
6,14616678358,2025-05-27 19:26:37+00:00,872.7,533,10.0,Run,1.637,2.653,3
7,14616638968,2025-05-27 19:08:25+00:00,3486.0,1045,37.3,Run,3.336,6.300,3
8,14606351709,2025-05-26 19:16:56+00:00,7250.5,2610,42.3,Run,2.778,5.353,3
9,14545227576,2025-05-20 15:37:53+00:00,6284.0,2316,41.1,Run,2.713,5.520,3


In [81]:
all_runs = pd.concat([my_runs, f1_runs, f2_runs, f3_runs], axis = 0)

In [83]:
all_runs.to_csv("Runs.csv", index = False)